<center><h1>Ejercicios con MySQL</h1></center>


In [1]:
#import libraries mysql
import mysql.connector

In [2]:
# Configuración de la conexión
config = {
    'user': 'root',
    'password': 'IT123DS_',
    'host': 'localhost',
    'raise_on_warnings': True
}

In [4]:
def execute_sql_from_file(filename, cursor):
    with open(filename, 'r') as file:
        # Ignorar líneas que comienzan con guiones
        commands = [line for line in file if not line.startswith('--') and not line.startswith('---')]
        sql_commands = "".join(commands).split(';')
        for command in sql_commands:
            try:
                if command.strip() != '':
                    cursor.execute(command)
            except Exception as e:
                print(f"Error executing {command}: {str(e)}")

In [5]:
# Establecer la conexión
connection = mysql.connector.connect(**config)
cursor = connection.cursor(buffered=True)

**EJERCICIO 1** : A partir de los documentos adjuntos (estructura y datos), crea una base de datos con MySQL. Muestra las principales características del esquema creado y explica las diferentes tablas y variables existentes.

In [6]:
#Database creation and selection according to the files
try:
    cursor.execute("CREATE DATABASE IF NOT EXISTS SGAE_Practica;")
except mysql.connector.Error as err:
    if err.errno == 1007:  # Error code for database exists
        print("La base de datos ya existe. Continuando...")
    else:
        raise

# Ejecutar instrucciones SQL de los archivos
execute_sql_from_file('Estructura_de_la_base_de_dades.txt', cursor)
execute_sql_from_file('Dades_a_introduir.txt', cursor)

In [16]:
#print table movies
cursor.execute("USE movies;")
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
for table in tables:
    print(table)

('tb_genre',)
('tb_movie',)
('tb_movie_person',)
('tb_person',)
('tb_role',)


*Descripción*: Se crea una base de datos llamada `movies` y se crean las tablas que corresponden al genre, movie, movie_person, y role.


In [18]:
#Selecciona la base de datos
cursor.execute("USE movies;")

#Muestra las tablas en esa base de datos
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

#Itera sobre cada tabla y muestra su contenido
for (table_name,) in tables:
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    #muestra informacion de las columnas de la tabla
    print(f"Tabla: {table_name}")
    print(cursor.column_names)


Tabla: tb_genre
('genre_id', 'genre_name', 'created_by_user', 'created_date', 'updated_date')
Tabla: tb_movie
('movie_id', 'movie_title', 'movie_date', 'movie_format', 'movie_genre_id', 'created_by_user', 'created_date', 'updated_date')
Tabla: tb_movie_person
('movie_id', 'person_id', 'role_id', 'movie_award_ind', 'created_by_user', 'created_date', 'updated_date')
Tabla: tb_person
('person_id', 'person_name', 'person_country', 'person_dob', 'person_dod', 'person_parent_id', 'created_by_user', 'created_date', 'updated_date')
Tabla: tb_role
('role_id', 'role_name', 'created_by_user', 'created_date', 'updated_date')


*Descripción*:  Se muestran las columnas que continen las tablas creadas.

**EJERCICIO 2**: Realiza la siguiente consulta sobre la recién creada base de datos: 

Tienes que obtener el nombre, el país y la fecha de nacimiento de aquellas personas por las que no conste una fecha de muerte y ordenar los datos de la persona más vieja a la persona más joven.

In [26]:
#muestra el nombre, pais y fecha de nacimiento de las personas que no tienen fecha de muerte y ordena los datos de la persona mas vieja a la mas joven en la tabla tb_person
cursor.execute("SELECT person_name, person_country, person_dob FROM tb_person WHERE person_dod IS NULL ORDER BY person_dob ASC")
rows = cursor.fetchall()
print("Nombre, pais y fecha de nacimiento de las personas que no tienen fecha de muerte y ordena los datos de la persona mas vieja a la mas joven en la tabla tb_person")
for row in rows:
    print(row)

Nombre, pais y fecha de nacimiento de las personas que no tienen fecha de muerte y ordena los datos de la persona mas vieja a la mas joven en la tabla tb_person
('John Williams', 'United States', datetime.date(1928, 8, 8))
('Vera Miles', 'United States', datetime.date(1929, 8, 23))
('Sean Connery', 'Scotland', datetime.date(1930, 7, 8))
('Robert Duvall', 'United States', datetime.date(1931, 1, 5))
('Morgan Freeman', 'United States', datetime.date(1935, 10, 1))
('Francis Ford Coppola', 'United States', datetime.date(1939, 4, 7))
('Gary Kurtz', 'United States', datetime.date(1940, 7, 27))
('Martin Sheen', 'United States', datetime.date(1940, 8, 3))
('Harrison Ford', 'United States', datetime.date(1942, 7, 13))
('George Lucas', 'United States', datetime.date(1944, 5, 14))
('Carmine Coppola', 'United States', datetime.date(1945, 7, 8))
('Steven Spielberg', 'United States', datetime.date(1946, 12, 18))
('Tim Robbins', 'United States', datetime.date(1949, 6, 7))
('Mel Gibson', 'Australia', d

**EJERCICIO 3**: Realiza la siguiente consulta sobre la recién creada base de datos: 

Tienes que obtener el nombre del género y el número total de películas de ese género y ordenarlo por orden descendente de número total de películas.  

In [30]:
#obteniendo el nombre del genero y el numero total de peliculas que pertenecen a ese genero
query = """
SELECT genre_name, COUNT(*) 
FROM tb_genre 
JOIN tb_movie ON tb_genre.genre_id = tb_movie.movie_genre_id 
GROUP BY genre_name
ORDER BY COUNT(*) DESC;
"""
cursor.execute(query)

# Recupera y muestra los resultados
results = cursor.fetchall()
for row in results:
    print(row)

('Comedia', 3)
('Acción', 2)
('Ciencia Ficción', 2)
('Drama', 2)
('Romance', 2)
('Suspense', 2)
('Fantasía', 1)
('Terror', 1)
('Bélico', 1)


**EJERCICIO 4**: Realiza la siguiente consulta sobre la recién creada base de datos: 

Tienes que obtener, para cada persona, su nombre y el número máximo de roles distintos que ha tenido en una misma película. 

Posteriormente, muestra únicamente a aquellas personas que hayan asumido más de un rol en una misma película.

In [31]:
# Ejecuta la consulta
query = """
SELECT p.person_name, MAX(sub.num_roles) AS max_roles_in_one_movie
FROM tb_person AS p
JOIN (
    SELECT movie_id, person_id, COUNT(DISTINCT role_id) AS num_roles
    FROM tb_movie_person
    GROUP BY movie_id, person_id
    HAVING num_roles > 1
) AS sub ON p.person_id = sub.person_id
GROUP BY p.person_name
HAVING max_roles_in_one_movie > 1
ORDER BY max_roles_in_one_movie DESC;
"""

cursor.execute(query)

# Recupera y muestra los resultados
results = cursor.fetchall()
for row in results:
    print(row)


('Francis Ford Coppola', 3)
('Alfred Joseph Hitchcock', 3)
('George Lucas', 2)
('Steven Spielberg', 2)
('Daniel Sanchez Arevalo', 2)
('Mel Gibson', 2)


**EJERCICIO 5**: Realiza la siguiente operación sobre la base de datos recién creada: 

Debes crear un nuevo género llamado "Documental" que tenga como identificador el número 69.

In [32]:
cursor.execute("INSERT INTO tb_genre (genre_id, genre_name) VALUES (69, 'Documental');")
connection.commit()

In [34]:
#muestra el nuevo genero agregado
cursor.execute("SELECT * FROM tb_genre WHERE genre_id = 69")
rows = cursor.fetchall()
print("Nuevo genero agregado")
for row in rows:
    print(row)

Nuevo genero agregado
(69, 'Documental', 'OS_SGAD', None, None)


**EJEERCICIO 6**: Realiza la siguiente operación sobre la base de datos recién creada:  

Elimina la película "La Gran Familia Española" de la base de datos.

In [35]:
#buscammos el id de la pelicula que queremos modificar
cursor.execute("SELECT movie_id FROM tb_movie WHERE movie_title = 'La Gran Familia Española';")
movie_id_to_delete = cursor.fetchone()
print("id de la pelicula que queremos modificar", movie_id_to_delete)

id de la pelicula que queremos modificar (11,)


In [38]:
# Primero, eliminamos las entradas relacionadas de tb_movie_person
if movie_id_to_delete:
    cursor.execute(f"DELETE FROM tb_movie_person WHERE movie_id = {movie_id_to_delete[0]};")
    connection.commit()

    # Luego, eliminamos la película
    cursor.execute(f"DELETE FROM tb_movie WHERE movie_id = {movie_id_to_delete[0]};")
    connection.commit()

In [39]:
#buscamos la pelicula eliminada
cursor.execute("SELECT * FROM tb_movie WHERE movie_title = 'La Gran Familia Española';")

**EJERCICIO 7**: Realiza la siguiente operación sobre la base de datos recién creada: 

Cambia el género de la película "Ocho apellidos catalanes" para que conste como comedia y no como romántica.

In [40]:
# Primero, obtenemos el genre_id para "Comedia"
cursor.execute("SELECT genre_id FROM tb_genre WHERE genre_name = 'Comedia';")
genre_id_comedia = cursor.fetchone()
if not genre_id_comedia:
    print("El género Comedia no se encuentra en la base de datos.")

# Luego, actualizamos el registro de la película "Ocho apellidos catalanes"
cursor.execute(f"UPDATE tb_movie SET movie_genre_id = {genre_id_comedia[0]} WHERE movie_title = 'Ocho apellidos catalanes';")
connection.commit()

print("Género de la película 'Ocho apellidos catalanes' actualizado correctamente.")


Género de la película 'Ocho apellidos catalanes' actualizado correctamente.


In [41]:
#mostramos ahora la pelicula con el nuevo genero
cursor.execute("SELECT * FROM tb_movie WHERE movie_title = 'Ocho apellidos catalanes';")
rows = cursor.fetchall()
print("Pelicula con el nuevo genero")
for row in rows:
    print(row)

Pelicula con el nuevo genero
(9, 'Ocho Apellidos Catalanes', datetime.date(2016, 6, 9), 'Digital', 3, 'OS_SGAD', None, None)


In [43]:
#miramos a quien pertenece el id 3 en tb_genre
cursor.execute("SELECT * FROM tb_genre WHERE genre_name = 'comedia';")
rows = cursor.fetchall()
print("A quien pertenece el id 9 en tb_genre")
for row in rows:
    print(row)

A quien pertenece el id 9 en tb_genre
(3, 'Comedia', 'OS_SGAD', None, None)


In [44]:
#close connection
cursor.close()
connection.close()